<!--TUTORIAL INFORMATION-->
<img align="left" style="padding-right:10px;" src="images/mindboggle_logo_200px_600dpi.jpg">
*This notebook provides a tutorial to the [Mindboggle software](http://mindboggle.info) and assumes that the Mindboggle docker container has been installed as per the [installation instructions](http://mindboggle.readthedocs.io/en/latest/#installing-mindboggle). In addition to showing examples of how to run Mindboggle on the command line, we will dig into the Python library to run individual functions (by [Arno Klein](http://binarybottle.com), [CC-BY](https://creativecommons.org/licenses/by/3.0/us/legalcode)).*

# Mindboggle tutorial

Mindboggle’s open source brain morphometry platform takes in preprocessed T1-weighted MRI data, and outputs volume, surface, and tabular data containing label, feature, and shape information for further analysis. In this tutorial, we will:

  - Look at Mindboggle's processing steps and output.
  - Run "mindboggle --help" on the command line in a Docker container.
  - Run "mindboggle" on the command line with example data.
  - Run individual functions within the mindboggle Python library.


## Mindboggle processing steps

The following steps are performed by Mindboggle:

  - Create hybrid gray/white segmentation from FreeSurfer and ANTs output.

  - Fill hybrid segmentation with FreeSurfer- or ANTs-registered labels.

  - Compute volume shape measures for each labeled region:
    - volume (volume_per_brain_region)
    - thickness of cortical labels (thickinthehead)

  - Compute surface shape measures for every cortical mesh vertex:
    - surface area
    - travel depth
    - geodesic depth
    - mean curvature
    - convexity (from FreeSurfer)
    - thickness (from FreeSurfer)
  
  - Extract cortical surface features:
    - folds
    - sulci
    - fundi

  - For each cortical surface label/sulcus, compute:
    - area
    - mean coordinates: means_per_label
    - mean coordinates in MNI152 space
    - Laplace-Beltrami spectrum
    - Zernike moments
  
  - Compute statistics (stats_per_label in compute.py) for each shape measure in 4 above for each label/feature:
    - median
    - median absolute deviation
    - mean
    - standard deviation
    - skew
    - kurtosis
    - lower quartile
    - upper quartile

Open Mindboggle's website summarizing the software's processing steps and output file tree.

In [ ]:
import webbrowser
url = 'http://mindboggle.readthedocs.io/en/latest/#mindboggle-processing-steps'
webbrowser.open(url)

## Mindboggle output

By default, output files are saved in $HOME/mindboggled/SUBJECT, where $HOME is the home directory and SUBJECT is a name representing the person’s brain that has been scanned. Volume files are in NIfTI format, surface meshes in VTK format, and tables are comma-delimited. Each file contains integers that correspond to anatomical labels or features (0-24 for sulci). All output data are in the original subject’s space. The following include outputs from most, but not all, optional arguments.

**Folder -- Contents -- Format**

**labels**/	-- number-labeled surfaces and volumes -- .vtk, .nii.gz

**features**/  -- surfaces with features: sulci, fundi -- .vtk

**shapes**/ -- surfaces with shape measures (per vertex) -- .vtk

**tables**/	-- tables of shape measures (per label/feature/vertex)	-- .csv

## "mindboggle --help" on the command line

First, let's see what command-line options Mindboggle provides:

In [ ]:
! mindboggle --help

## "mindboggle" command with example data

In the following example, we will run Mindboggle on FreeSurfer (but not ANTs) data to get shape measures such as curvature, depth, and area for cortical surface labels, and to make it quick, we will turn off volume labels and shape measures, and surface-based Zernike moments and Laplace-Beltrami spectra.

In [ ]:
%%bash
HOST=/root/data/Users/arno;
FREESURFER_SUBJECT=$HOST/mindboggle_input_example/freesurfer/subjects/arno;
ANTS_SUBJECT=$HOST/mindboggle_input_example/ants/subjects/arno;
MINDBOGGLING=$HOST/mindboggling;
MINDBOGGLED=$HOST/mindboggled;

mindboggle $FREESURFER_SUBJECT --working $MINDBOGGLING --out $MINDBOGGLED \
           --no_volumes --no_sulci --no_points --no_moments --no_spectra

## Mindboggle library

Rather than call Mindboggle from the command line, we can also call individual Python functions within the Mindboggle library, which includes the following files:

  - mindboggle/mindboggle/
    - **mio**/  *-- input/output functions*
        - **colors**.py  *-- colormap-related functions*
        - **convert_volumes**.py  *-- read/write nifti volume files*
        - **fetch_data**.py  *-- fetch data from a URL or from third party software*
        - **labels**.py  *-- information about labeling protocols*
        - **plots**.py  *-- plot functions*
        - **tables**.py  *-- read/write tables*
        - **vtks**.py  *-- read/write VTK surface files*
    - **guts**/  *-- the "guts" underlying feature extraction and labeling code*
        - **compute**.py  *-- compute distances, etc.*
        - **graph**.py  *-- graph operations*
        - **kernels**.py  *-- kernels for graph operations*
        - **mesh**.py  *-- operate on surface mesh vertices*
        - **paths**.py  *-- connect surface mesh vertices*
        - **rebound**.py  *-- adjust label borders on a surface mesh*
        - **relabel**.py  *-- relabel surface or volume files*
        - **segment**.py  *-- segment a surface mesh*
    - **features**/  *-- higher-level feature extraction (folds, fundi, sulci)*
        - **folds**.py  *-- extract surface folds*
        - **fundi**.py  *-- extract fundus curves from folds*
        - **sulci**.py  *-- extract sulci from folds*
    - **shapes**/  *-- shape measurement functions
        - **surface_shapes**.py  *-- compute surface shapes (calls C++ library below)*
        - **volume_shapes**.py  *-- compute volumes and thicknesses*
        - **laplace_beltrami**.py  *-- compute a Laplace-Beltrami spectrum*
        - **zernike/zernike**.py  *-- compute Zernike moments of a collection of vertices*
        - **likelihood**.py  *-- compute (fundus) likelihood values*
    - **thirdparty/**  *-- third-party code*
        - **ants**.py  *-- call ANTs commands*
        - **vtkviewer**.py  *-- VTK viewer (by Hal Canary)*

  - mindboggle/vtk_cpp_tools  *-- C++ tools for measuring shapes on VTK surface files*
    - **area**/
    - **curvature**/
    - **geodesic_depth**/
    - **gradient**/
    - **surface_overlap**/
    - **travel_depth**/

## Python functions

Let's run some functions within Mindboggle's Python library.  The following examples are excerpts from the above files' docstrings.

In [ ]:
# Compute histogram values from nibabel-readable image:
from mindboggle.guts.compute import compute_image_histogram
from mindboggle.mio.fetch_data import prep_tests
urls, fetch_data = prep_tests()
labels_file = fetch_data(urls['freesurfer_labels'], '', '.nii.gz')
nbins = 100
threshold = 0.5
histogram_values = compute_image_histogram(labels_file, nbins, threshold)
histogram_values

### Example:
Compute various statistical measures across vertices per label, optionally using weights (such as surface area per vertex).

In [ ]:
"""
    Example (area-weighted mean):
    average value = sum(a_i * v_i) / total_surface_area,
    where *a_i* and *v_i* are the area and value for each vertex *i*.

    Reference:
        Weighted skewness and kurtosis unbiased by sample size
        Lorenzo Rimoldini, arXiv:1304.6564 (2013)
        http://arxiv.org/abs/1304.6564

    Note ::
        This function is different than means_per_label() in two ways:
            1. It computes more than simply the (weighted) mean and sdev.
            2. It only accepts 1-D arrays of values.

    Parameters
    ----------
    values : numpy array of individual or lists of integers or floats
        values for all vertices
    labels : list or array of integers
        label for each value
    include_labels : list of integers
        labels to include
    exclude_labels : list of integers
        labels to be excluded
    weights : numpy array of floats
        weights to compute weighted statistical measures
    precision : integer
        number of decimal places to consider weights

    Returns
    -------
    medians : list of floats
        median for each label
    mads : list of floats
        median absolute deviation for each label
    means : list of floats
        mean for each label
    sdevs : list of floats
        standard deviation for each label
    skews : list of floats
        skew for each label
    kurts : list of floats
        kurtosis value for each label
    lower_quarts : list of floats
        lower quartile for each label
    upper_quarts : list of floats
        upper quartile for each label
    label_list : list of integers
        list of unique labels
"""
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np
from mindboggle.mio.vtks import read_scalars
from mindboggle.guts.compute import stats_per_label
from mindboggle.mio.fetch_data import prep_tests
urls, fetch_data = prep_tests()
values_file = fetch_data(urls['left_mean_curvature'], '', '.vtk')
labels_file = fetch_data(urls['left_freesurfer_labels'], '', '.vtk')
area_file = fetch_data(urls['left_area'], '', '.vtk')
values, name = read_scalars(values_file, True, True)
areas, name = read_scalars(area_file, True, True)
labels, name = read_scalars(labels_file)
include_labels = []
exclude_labels = [-1]
weights = areas
precision = 1
medians, mads, means, sdevs, skews, kurts, lower_quarts, upper_quarts, label_list = stats_per_label(values, 
                                                                                                    labels, include_labels, exclude_labels, weights, precision)
[np.float("{0}:.{1}f {2}:.{1}f {3}:.{1}f {4}:.{1}f".format(medians[i], 5, mads[i], means[i], kurts[i]) for i in range(len(mads))]